### First Practice Activity of a Recommendation System

To begin, my objective is learn about RecSystems with Reinforcement Learning, but as i don't know anything about RecSys or RL, i decided to understanding each term separately and, after that, finally study the two as a unique theme. Therefore, i will start with Recommendation System, due to the fact of be more simple than RL, beginning with all important definitions in these area of study.

#### First Ideas
The basis of the recommendation system is "What do you want to recommend, for whom, and how?" you can find many ways to make recommendations.

For example, if you have a client who watches a horror movie, you have two main ways to choose a recommendation for him. The first is to recommend other horror movies like the first one, i.e., similar items(films) to the original article in the interaction—that is an item-focused approach.

In other cases, you wish to recommend movies watched by other users similar to the client who watches that horror movie; in that case, you use a User-focused approach.

You can apply these approaches to different types of recommendation systems.


Despite that, you have two ways to make a recommendation system: collaborative and content.

##### Collaborative

Collaborative Methods use past interactions between users and items to make new recommendations using a "User-Item Interactions Matrix." In the middle of the recommendation, it uses the proximity between the users/items to predict. Is it possible to use a model-based approach or a metric to calculate the similarity and just use this metric.

It has some issues, like problems with new Users and Items, but with other methods specific to further information that can surpass this problem.

A lot of methods using a collaborative approach, such as:

        - Memory Based
        - Model-Based

##### Content

Content methods use additional information about users and/or items. In the movie example, you can have information about the Release Year, the Director, the Principal Actors, and many other data.

These methods have much more information and don't suffer with new Users or Items too much because they 

### Practice

I will use the most simple recommendation system for this first recommendation system, a memory-based collaborative model. These recommendation systems use just similarity metrics to make a recommendation, i.e., they don't have any ML model in the middle of the process as the simplest method is the most interesting way to new students like me.

One of the problems of that method is the bad scalability with the new users and new movies, because of the pivot table.

How i gonna use a collaborative approach, i need just the information about the interactions user-item.

I include the movie-title to better visualization of the recommendations in the final of this notebook.

In [21]:
import numpy as np
from scipy.stats import pearsonr
from scipy.sparse import csr_matrix, csc_matrix
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

In [22]:
names = pd.read_csv("./movie_information/movie.csv")

display(names.info())

interactions = pd.read_csv("./movie_information/rating.csv")

display(interactions.info())

print()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27278 entries, 0 to 27277
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   movieId  27278 non-null  int64 
 1   title    27278 non-null  object
 2   genres   27278 non-null  object
dtypes: int64(1), object(2)
memory usage: 639.5+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000263 entries, 0 to 20000262
Data columns (total 4 columns):
 #   Column     Dtype  
---  ------     -----  
 0   userId     int64  
 1   movieId    int64  
 2   rating     float64
 3   timestamp  object 
dtypes: float64(1), int64(2), object(1)
memory usage: 610.4+ MB


None

In [23]:
del interactions["timestamp"]

del names["genres"]

# Removing irrelevant information

Merging the two dataframes in one

In [24]:
data = pd.merge(names, interactions)

data.head()

,movieId,title,userId,rating
0,1,Toy Story (1995),3,4.0
1,1,Toy Story (1995),6,5.0
2,1,Toy Story (1995),8,4.0
3,1,Toy Story (1995),10,4.0
4,1,Toy Story (1995),11,4.5


Taking a sample of 10 Million to reduce the dimensionality of the DataFrame

In [25]:
sampled_data = data.iloc[:10000000, :]

Removing users with low count of reviews


In [26]:
users_count = sampled_data["userId"].value_counts()

print(users_count[-100:-1]) # a lot of users with low count of viewers

filtered_index = sampled_data.index[sampled_data.index.isin(users_count[users_count < 20].index)].tolist()

filtered_sampled_data = sampled_data.drop(index = filtered_index, axis = 1)

84323     1
53853     1
90167     1
25740     1
113543    1
         ..
55622     1
104829    1
98733     1
126596    1
113038    1
Name: userId, Length: 99, dtype: int64


In [27]:
movie_counts = sampled_data["movieId"].value_counts()

filtered_index = sampled_data.index[sampled_data.index.isin(movie_counts[movie_counts < 30].index)].tolist()

filtered_sampled_data = sampled_data.drop(index = filtered_index, axis = 1)

Creating a pivot table with the format: 
    
    - Rows: Users
    - Columns: Items

With this pivot we can make the recommendations.

In [28]:
# Tables used in collaborative approaches have the format: Rows = users, Columns = Items, so we need to create a Pivot table with our data

pivot_df = filtered_sampled_data.pivot_table(index = ["userId"], columns = ["title"], values = "rating").fillna(0)

# After some tests use the fillna(0) bring better results than maintain the NA

pivot_df

title,'Til There Was You (1997),"'burbs, The (1989)",1-900 (06) (1994),101 Dalmatians (1996),101 Dalmatians (One Hundred and One Dalmatians) (1961),12 Angry Men (1957),187 (One Eight Seven) (1997),1984 (Nineteen Eighty-Four) (1984),2 Days in the Valley (1996),2 ou 3 choses que je sais d'elle (2 or 3 Things I Know About Her) (1967),...,"Yes, Madam (a.k.a. Police Assassins) (a.k.a. In the Line of Duty 2) (Huang gu shi jie) (1985)",You Can't Take It with You (1938),Young Frankenstein (1974),Young Guns (1988),Young Guns II (1990),"Young Poisoner's Handbook, The (1995)",Your Friends and Neighbors (1998),Zero Effect (1998),Zero Kelvin (Kjærlighetens kjøtere) (1995),Zeus and Roxanne (1997)
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,...,0.0,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
138489,0.0,0.0,0.0,0.0,0.0,4.5,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138490,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
138491,0.0,0.0,0.0,0.0,2.5,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
movie_names = pivot_df.columns.to_series().reset_index(drop = True)

print(movie_names)

pivot_array = np.array(pivot_df) # We will use a array to make the computation more efficient

0                               'Til There Was You (1997)
1                                      'burbs, The (1989)
2                                       1-900 (06) (1994)
3                                   101 Dalmatians (1996)
4       101 Dalmatians (One Hundred and One Dalmatians...
                              ...                        
2079                Young Poisoner's Handbook, The (1995)
2080                    Your Friends and Neighbors (1998)
2081                                   Zero Effect (1998)
2082           Zero Kelvin (Kjærlighetens kjøtere) (1995)
2083                              Zeus and Roxanne (1997)
Name: title, Length: 2084, dtype: object


In [30]:
pivot_csr = csr_matrix(pivot_array)

pivot_csc = csc_matrix(pivot_array)

# Creating two sparses tables to make a more efficient code

In [31]:
def correlation_pearson_sparse_row(array, sparse_matrix):
    
    yy = array - array.mean() 
    xm = sparse_matrix.mean(axis=1).A.ravel() # pegando a média
    ys = yy / np.sqrt(np.dot(yy, yy)) # calculando ys
    xs = np.sqrt(np.add.reduceat(sparse_matrix.data**2, sparse_matrix.indptr[:-1]) - sparse_matrix.shape[1]*xm*xm)

    correl2 = np.add.reduceat(sparse_matrix.data * ys[sparse_matrix.indices], sparse_matrix.indptr[:-1]) / xs
    return correl2

In [32]:
def correlation_pearson_sparse_column(array, sparse_matrix):
    
    yy = array - array.mean() 
    xm = sparse_matrix.mean(axis=0).A.ravel() # pegando a média
    ys = yy / np.sqrt(np.dot(yy, yy)) # calculando ys
    xs = np.sqrt(np.add.reduceat(sparse_matrix.data**2, sparse_matrix.indptr[:-1]) - sparse_matrix.shape[0]*xm*xm)

    correl2 = np.add.reduceat(sparse_matrix.data * ys[sparse_matrix.indices], sparse_matrix.indptr[:-1]) / xs
    return correl2

In [33]:
def search_movie_name(search):
   if search in movie_names:
      return search
   result = (movie_names.loc[movie_names.str.contains(search)]).reset_index(drop = True)

   if result.empty:
      print("No Movies found.")
      return None

   return result.loc[0]

In [34]:
def get_recommendation_item(movie_name, metric):
    if (movie_name is None) or (movie_names.str.match(movie_name).any()):
        print("Movie not found.")
        return None
    
    movie = pivot_array[:, pd.Index(movie_names).get_loc(movie_name)]

    if metric == "cosine":
        similarity = cosine_similarity(movie.reshape(1, -1), pivot_csc.T)[0]
    elif metric == "pearson":
        similarity = correlation_pearson_sparse_column(movie, pivot_csc)
    else: return None
    
    order = np.argsort(-similarity)

    top_10 = similarity[order[1:11]]
    top_10_names = movie_names[order[1:11]]
    
    return (list(top_10_names), top_10)

In [35]:
movie_name = search_movie_name("Batman")

result = get_recommendation_item(movie_name, "cosine")

print("Recommendations(Cosine Similarity) for the Movie: ", movie_name)

if not(movie_name is None):
    display(pd.DataFrame(result[0]))

Recommendations(Cosine Similarity) for the Movie:  Batman & Robin (1997)


,0
0,Batman Returns (1992)
1,"Lost World: Jurassic Park, The (1997)"
2,Con Air (1997)
3,Lethal Weapon 3 (1992)
4,Armageddon (1998)
5,Die Hard 2 (1990)
6,Men in Black (a.k.a. MIB) (1997)
7,Lethal Weapon 4 (1998)
8,Air Force One (1997)
9,Face/Off (1997)


In [36]:
movie_name = search_movie_name("Batman")

result = get_recommendation_item(movie_name, "pearson")

print("Recommendations(Pearson Correlation) for the Movie: ", movie_name)

if not(movie_name is None):
    display(pd.DataFrame(result[0]))

Recommendations(Pearson Correlation) for the Movie:  Batman & Robin (1997)


,0
0,Batman Returns (1992)
1,"Lost World: Jurassic Park, The (1997)"
2,Con Air (1997)
3,Lethal Weapon 3 (1992)
4,Lethal Weapon 4 (1998)
5,Godzilla (1998)
6,Die Hard 2 (1990)
7,Armageddon (1998)
8,Air Force One (1997)
9,Tomorrow Never Dies (1997)


##### Consideration about cosine similarity

A important information is cosine similarity dont considers magnitude, i.e. if a user rate all films they see with 0, is perfectly similar to another user who rates all the same films as 5, is important considers the rating of a film, but the users are similars in some way, when you focus in watched movies.

Cosine Similarity can be a bit dangerous, but if you know the risks, you can consider using this metric.


### User-Based


In [37]:
def get_recommendation_user(user_id, metric):
    first_user = pivot_array[user_id, :]
    watched_movies_index = np.where(first_user > 0.)[0]

    if metric == "cosine":
        similarity_table = cosine_similarity(first_user.reshape(1, -1), pivot_csr)[0]
    elif metric == "pearson":
        similarity_table = correlation_pearson_sparse_row(first_user, pivot_csr)
    else: 
        print("Unknown Metric.")
        return None
    # Calc similarity table

    num_columns = pivot_csc.shape[1] # take num_movie
    
    # Creating the two Results for the ranking
    total = np.zeros(num_columns)
    similaritySums = np.zeros(num_columns)

    for col in range(num_columns):
        if col in watched_movies_index: continue
        column_data = pivot_csc[:, col].T # quero uma matriz linha
        # For each unwatched movie we calculate the (Similarity * Rating) Sum and the Similarity Sum 
        total[col] = column_data.dot(similarity_table)

        column_data.data = np.ones(column_data.data.shape)
        similaritySums[col] = column_data.dot(similarity_table)

    # the division between the total and similaritySums result in the "Predicted" Rate for that movie, calculated using the rate of the observed Rate
    similaritySums_smoothed = similaritySums + 1e-10 # to avoid division by 0
    index = (np.argsort(-(total/similaritySums_smoothed)))
    values = -(np.sort(-(total/similaritySums_smoothed)))
    return index, values

## Optimized

In [38]:
index, values = get_recommendation_user(0, "cosine")
pd.DataFrame(movie_names[index])[:10]

,title
755,"Godfather, The (1972)"
756,"Godfather: Part II, The (1974)"
1650,Seven Samurai (Shichinin no samurai) (1954)
1627,Schindler's List (1993)
529,Dr. Strangelove or: How I Learned to Stop Worr...
1544,Rear Window (1954)
1809,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
326,Casablanca (1942)
5,12 Angry Men (1957)
1434,Paths of Glory (1957)


In [39]:
index, values = get_recommendation_user(0, "pearson")
pd.DataFrame(movie_names[index])[:10]

,title
1160,MURDER and murder (1996)
755,"Godfather, The (1972)"
756,"Godfather: Part II, The (1974)"
1650,Seven Samurai (Shichinin no samurai) (1954)
1627,Schindler's List (1993)
529,Dr. Strangelove or: How I Learned to Stop Worr...
1544,Rear Window (1954)
5,12 Angry Men (1957)
1809,Sunset Blvd. (a.k.a. Sunset Boulevard) (1950)
326,Casablanca (1942)


In [40]:
pivot_array.shape
# 2084 Movies and 138300 Users

(138300, 2084)

In the future, testing other metrics, also other approaches as a Content-Model 

#### OK